## Import Library

meghubungkan ke google drive

In [43]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)

Mounted at /content/drive


In [44]:
import pandas as pd
import numpy as np

# Preprocessing
import re, string
from spacy.lang.id import Indonesian
import spacy

# Prediksi Klasifikasi
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
import joblib

# Deteksi Objek
import os
import cv2
import torch
from PIL import Image

## Dataset

In [45]:
df_train = pd.read_csv("/content/drive/MyDrive/skripsi/teks/X_train_baru_2.csv")

In [46]:
dfJson = pd.read_json("/content/drive/MyDrive/skripsi/test/JSON/Buku Kas Umum/BKU Agustus.json")
dfJsonT = dfJson.transpose()

In [47]:
dfJsonT

,text,image_path
page_1,BUKU KAS UMUM\n\nBulan : Agustus 2021\n\nTangg...,images/New\BKU Agustus_1.jpeg
page_2,Bulan : Agustus 2021\n\nTanggal Cetak: 01-02-2...,images/New\BKU Agustus_2.jpeg
page_3,BUKU KAS UMUM\n\nBulan : Agustus 2021\n\n31-08...,images/New\BKU Agustus_3.jpeg


In [48]:
# menggabungkan beberapa halaman pada dokumen menjadi satu baris dataframe
dfJsonBaru = ''.join(dfJsonT.text)
imagePath = ''.join(dfJsonT.image_path)
dt = {'teks' : [dfJsonBaru],
      'imgPath': [imagePath]}
data = pd.DataFrame(dt)

In [49]:
data

,teks,imgPath
0,BUKU KAS UMUM\n\nBulan : Agustus 2021\n\nTangg...,images/New\BKU Agustus_1.jpegimages/New\BKU Ag...


In [51]:
# mengecek isi dokumen apakah kosong atau ada isinya untuk keseluruhan dokumen
import sys
if data['teks'][0] == '':
  sys.exit("Dokumen tidak valid. Mohon unggah dokumen yang sesuai")

## Preprocessing Data Input Dokumen

*cleaning text*

In [52]:
def casefolding(text):
  text = text.lower()
  text = text.replace("_<.*?/\|-_:^()>", ' ')
  text = text.strip()
  text = re.compile('<.*?/\|-_:^()>——').sub('', text)
  text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)
  text = text.translate(str.maketrans("","",string.punctuation))
  text = re.sub('\+s', ' ', text)
  text = re.sub(r'\s+', ' ', text)
  text = text.replace("\n", ' ')
  text = text.replace("/\-———————— « — —", ' ')
  text = text.encode('ascii', 'ignore').decode('ascii')
  text = re.sub(r'[^\x00-\x7f]', "", text)

  return text

*tokenize*

In [53]:
def tokenisasi(text):
  tokens = text.split(" ")
  return tokens

*stopword removal/stopword elimination*

In [54]:
def stopword_elim(text):
    nlp = Indonesian()
    nlp = spacy.blank('id')
    stopwords = nlp.Defaults.stop_words
    tokens = tokenisasi(text)
    tokens_nostopword = [w for w in tokens if not w in stopwords]
    return (" ").join(tokens_nostopword)

Memanggil function masing-masing preprocessing

In [55]:
def preprocess(string):
  return stopword_elim(casefolding(string))

In [56]:
data['clean_text'] = data['teks'].apply(lambda x: preprocess(x))

In [57]:
data

,teks,imgPath,clean_text
0,BUKU KAS UMUM\n\nBulan : Agustus 2021\n\nTangg...,images/New\BKU Agustus_1.jpegimages/New\BKU Ag...,buku kas agustus 2021 tanggal cetak 01 02 2022...


## Prediksi Klasifikasi

In [58]:
X_train = np.array(df_train['0'].values.astype('U'))

In [59]:
X_baru = np.array(data['clean_text'].values.astype('U'))

*Feature selection* menggunakan pembobotan TF-IDF

In [60]:
tfidf_vectorizer = TfidfVectorizer(use_idf = True, encoding='latin-1').fit(X_train)

X_baru_tfidf = tfidf_vectorizer.transform(X_baru).toarray()

Prediksi dengan model yang sudah ada

In [61]:
teks_path = "/content/drive/MyDrive/skripsi/teks/model_mnb_baru"

In [62]:
def prediksi(path):
  loaded_model = joblib.load(path)
  y_pred_mnb = loaded_model.predict(X_baru_tfidf)
  hasil = y_pred_mnb[0]
  return hasil

In [63]:
pred = prediksi(teks_path)

In [64]:
pred

4

In [65]:
# pengecekan jenis dokumen sudah sesuai atau belum
if pred != 4 :
  text = "Dokumen tidak valid. Mohon unggah dokumen yang sesuai"
  print(text)
  sys.exit()

## Deteksi Tanda Tangan dan Stempel

In [66]:
imgPath = [dfJson[x]["image_path"] for x in dfJson]

In [67]:
def fullpath(image_path):
  new_path = []
  for x in imgPath:
    path_full = "\content\drive\\MyDrive\\skripsi\\test\\image\\Buku Kas Umum\\" + x
    path_full = path_full.replace("\\", "/")
    new_path.append(path_full)
    img = cv2.imread(path_full)
  return new_path

In [68]:
path_list = fullpath(imgPath)

In [69]:
%cd /content/drive/MyDrive/skripsi/YOLOv5/Signature-Verification_System_using_YOLOv5-and-CycleGAN/Training/YOLOv5/yolov5

/content/drive/MyDrive/skripsi/YOLOv5/Signature-Verification_System_using_YOLOv5-and-CycleGAN/Training/YOLOv5/yolov5


In [70]:
!git clone https://github.com/ultralytics/yolov5
!cd yolov5 && pip install -r requirements.txt
!cd ..

fatal: destination path 'yolov5' already exists and is not an empty directory.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.0/605.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.5 MB/s eta 0:00:00


In [71]:
# memanggil model hasil training
fine_tune = torch.hub.load('ultralytics/yolov5', 'custom', path='runs/train/exp/weights/best.pt', force_reload=True)

/usr/local/lib/python3.10/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
WARNING ⚠️ 'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING ⚠️ 'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use

In [72]:
def detect_objects(path):
  detections = []
  i = 1
  # cek setiap gambar halaman
  for filename in path:
    if filename.endswith('jpeg'):
      image = Image.open(filename).convert("RGB")

      results = fine_tune(image)

      s = False
      d = False

      if len(results.xyxy[0]) == 0 :
        detection = {
            "Stamp" : s,
            "DLSignature" : d,
            "page"  : i
        }
        detections.append(detection)
      else:
        for result in results.xyxy[0]:
          xmin, ymin, xmax, ymax, confidence, class_id = result.tolist()

          if fine_tune.names[int(class_id)] == "Stamp":
            if confidence > 0.4:
              s = True
          if fine_tune.names[int(class_id)] == "DLSignature":
            if confidence > 0.4:
              d = True

        detection = {
              "Stamp" : s,
              "DLSignature" : d,
              "page"  : i
        }
        detections.append(detection)
      i+=1

  return detections

In [73]:
# deteksi objek pada dokumen dari path gambar
detect = detect_objects(path_list)

In [74]:
detect

[{'Stamp': False, 'DLSignature': False, 'page': 1},
 {'Stamp': False, 'DLSignature': False, 'page': 2},
 {'Stamp': False, 'DLSignature': False, 'page': 3}]

### Output

In [75]:
s = False
d = False
p = []

for i in detect:
  if i['Stamp'] == True :
    s = True
  if i['DLSignature'] == True:
    d = True
  if i['Stamp'] == True and i['DLSignature']:
    p.append(i['page'])

detection = {
              "Jenis" : pred,
              "Stamp" : s,
              "DLSignature" : d,
              "page" : p
}
text = ''
if detection['Jenis'] == 4:
  if detection['Stamp'] == True:
      if detection['DLSignature'] == True:
        pages = ', '.join(map(str, detection['page']))
        text = "Dokumen Valid. Dokumen merupakan Buku Kas Umum dan terdapat stempel dan tanda tangan" + " di halaman " + pages
      else:
        text = "Dokumen sesuai, namun hanya terdapat stempel. Mohon unggah dokumen kembali dengan tanda tangan"
  else:
      if detection['DLSignature'] == True:
        text = "Dokumen sesuai, namun hanya terdapat tanda tangan. Mohon unggah dokumen kembali dengan stempel"
      else:
        text = "Dokumen sesuai, namun tidak ditemukan tanda tangan dan stempel. Mohon unggah dokumen kembali"
else:
  text = "Dokumen tidak valid. Mohon unggah dokumen yang sesuai"

print(text)

Dokumen sesuai, namun tidak ditemukan tanda tangan dan stempel. Mohon unggah dokumen kembali
